In [ ]:
# DS776 Auto-Update (runs in ~2 seconds, only updates when needed)
# If this cell fails, see Lessons/Course_Tools/AUTO_UPDATE_SYSTEM.md for help
%run ../../Lessons/Course_Tools/auto_update_introdl.py

# Part 1 Tutorial: Understanding Named Entity Extraction

**This tutorial will help you understand the data structure and what you need to do for Part 1.**

If you've been struggling with Part 1, this notebook will walk you through it step by step with working code you can run and modify.

## Import Libraries

First, let's import everything we'll need:

In [ ]:
from datasets import load_dataset
from collections import defaultdict
import pandas as pd

## Step 1: Understanding What the Data Looks Like

Let's start by looking at ONE example from the dataset to understand the structure:

In [ ]:
# Load the dataset
dataset = load_dataset("hobbes99/fake_movie_reviews_ner_sentiment")
label_list = dataset["train"].features["ner_tags"].feature.names

# Look at the FIRST example
example = dataset["train"][0]
print("Full example:")
print(example)
print("\n" + "="*60)

### What does this mean?

The output shows a dictionary with three keys:
- **tokens**: A list of words in the review (one review broken into individual words)
- **ner_tags**: Numbers that represent the entity type for each word
- **sentiment**: 0 = negative, 1 = positive

### Understanding the `label_list`

The `label_list` is the "decoder" for the numbers in `ner_tags`:

In [ ]:
print("Label list (the decoder):")
print(label_list)
print("\nWhat each number means:")
for i, label in enumerate(label_list):
    print(f"  {i} → '{label}'")

Each number corresponds to a label:
- 0 → 'O' (Outside - not an entity)
- 1 → 'B-ACTOR' (Beginning of an actor's name)
- 2 → 'I-ACTOR' (Inside/continuation of an actor's name)
- 3 → 'B-DIRECTOR' (Beginning of a director's name)
- 4 → 'I-DIRECTOR' (Inside/continuation of a director's name)

### Let's decode the example to make it readable:

In [ ]:
# Show the example with decoded labels
tokens = example['tokens']
ner_tags = example['ner_tags']
sentiment = example['sentiment']

print("Let's decode this example:\n")
print(f"{'Token':<15} {'Tag (number)':<15} {'Tag (label)':<15}")
print("="*50)
for token, tag_num in zip(tokens, ner_tags):
    tag_label = label_list[tag_num]
    print(f"{token:<15} {tag_num:<15} {tag_label:<15}")

print("\nSentiment:", "Positive (1)" if sentiment == 1 else "Negative (0)")

**Can you see the pattern?**

When you see B-ACTOR followed by I-ACTOR, that means those tokens together form one actor's name!
- First token gets B-ACTOR (Begin)
- Following tokens get I-ACTOR (Inside/continuation)

Same logic for directors.

## Step 2: What Does "Extract" Mean?

**"Extract" means:** Find all the actors and directors from the review and save them as complete names.

From the example above, we want to **extract**:
- Complete actor names (combining B-ACTOR + I-ACTOR tokens)
- Complete director names (combining B-DIRECTOR + I-DIRECTOR tokens)
- The sentiment (positive or negative)

### Where do we store this information?

We'll create **dictionaries** to count how many times each actor/director appears in positive vs negative reviews.

In [ ]:
# Example of what we're building toward:
example_storage = {
    "Tom Hanks": {"positive": 5, "negative": 2},
    "Meryl Streep": {"positive": 3, "negative": 1}
}

print("This is the kind of structure we want:")
print(example_storage)
print("\nThis would mean:")
print("- Tom Hanks appeared in 5 positive reviews and 2 negative reviews")
print("- Meryl Streep appeared in 3 positive reviews and 1 negative review")

## Step 3: Understanding "Token Lists to Strings"

### What is a "token list"?

A **token list** is just the list of words that make up a name.

In [ ]:
# For an actor like "Tom Hanks"
token_list = ['Tom', 'Hanks']
print("Token list:", token_list)
print("Type:", type(token_list))

# Convert to a string by joining with spaces
name_string = ' '.join(token_list)
print("\nConverted to string:", name_string)
print("Type:", type(name_string))

### What about "consecutive I- tags with B- tag"?

This means when you see a B-ACTOR followed by I-ACTOR tags, they belong to the SAME person:

In [ ]:
# Example with multiple actors
tokens_example = ['Tom', 'Hanks', 'and', 'Rita', 'Wilson']
tags_example = ['B-ACTOR', 'I-ACTOR', 'O', 'B-ACTOR', 'I-ACTOR']

print(f"{'Token':<10} {'Tag':<10}")
print("="*25)
for token, tag in zip(tokens_example, tags_example):
    print(f"{token:<10} {tag:<10}")

print("\nWe should extract TWO actors:")
print("1. 'Tom Hanks' (B-ACTOR + I-ACTOR)")
print("2. 'Rita Wilson' (B-ACTOR + I-ACTOR)")

## Step 4: Let's Look at a Real Example in Detail

In [ ]:
# Let's examine the first example more carefully
example = dataset["train"][0]

tokens = example['tokens']
ner_tags = example['ner_tags']
sentiment = example['sentiment']

print("Full review:", ' '.join(tokens))
print("\nWord-by-word breakdown:")
print(f"{'Token':<15} {'Tag':<15} {'Meaning':<30}")
print("="*60)

for token, tag_num in zip(tokens, ner_tags):
    tag_label = label_list[tag_num]
    if tag_label.startswith('B-'):
        meaning = "START of " + tag_label[2:]
    elif tag_label.startswith('I-'):
        meaning = "CONTINUES " + tag_label[2:]
    else:
        meaning = "Not an entity"
    print(f"{token:<15} {tag_label:<15} {meaning:<30}")

print(f"\nSentiment: {'Positive' if sentiment == 1 else 'Negative'}")

## Step 5: Extracting Entities from One Example

Now let's write a function to extract actors and directors from ONE example:

In [ ]:
def extract_entities_from_one_example(example, label_list):
    """
    Extract actor and director names from one example.
    Returns a dictionary with 'actors' and 'directors' lists.
    """
    tokens = example['tokens']
    ner_tags = example['ner_tags']
    
    actors = []  # Will store complete actor names
    directors = []  # Will store complete director names
    
    current_entity = []  # Store tokens for current entity we're building
    current_type = None  # Is it ACTOR or DIRECTOR?
    
    for i in range(len(tokens)):
        token = tokens[i]
        tag = label_list[ner_tags[i]]  # Convert number to label string
        
        if tag == 'B-ACTOR':
            # Before starting new entity, save the previous one if it exists
            if current_entity and current_type == 'ACTOR':
                actors.append(' '.join(current_entity))
            elif current_entity and current_type == 'DIRECTOR':
                directors.append(' '.join(current_entity))
            
            # Start new actor
            current_entity = [token]
            current_type = 'ACTOR'
        
        elif tag == 'I-ACTOR':
            # Continue current actor name
            if current_type == 'ACTOR':
                current_entity.append(token)
        
        elif tag == 'B-DIRECTOR':
            # Save previous entity
            if current_entity and current_type == 'ACTOR':
                actors.append(' '.join(current_entity))
            elif current_entity and current_type == 'DIRECTOR':
                directors.append(' '.join(current_entity))
            
            # Start new director
            current_entity = [token]
            current_type = 'DIRECTOR'
        
        elif tag == 'I-DIRECTOR':
            # Continue current director name
            if current_type == 'DIRECTOR':
                current_entity.append(token)
        
        else:  # tag == 'O' (outside any entity)
            # Save previous entity
            if current_entity and current_type == 'ACTOR':
                actors.append(' '.join(current_entity))
            elif current_entity and current_type == 'DIRECTOR':
                directors.append(' '.join(current_entity))
            
            # Reset
            current_entity = []
            current_type = None
    
    # IMPORTANT: Don't forget the last entity!
    # If the review ends with an actor/director name, we need to save it
    if current_entity and current_type == 'ACTOR':
        actors.append(' '.join(current_entity))
    elif current_entity and current_type == 'DIRECTOR':
        directors.append(' '.join(current_entity))
    
    return {'actors': actors, 'directors': directors}


# Test it on the first example
example = dataset["train"][0]
entities = extract_entities_from_one_example(example, label_list)

print("Review:", ' '.join(example['tokens']))
print("\nExtracted entities:")
print("  Actors:", entities['actors'])
print("  Directors:", entities['directors'])
print("  Sentiment:", "positive" if example['sentiment'] == 1 else "negative")

Let's test on a few more examples to make sure it works:

In [ ]:
# Test on first 5 examples
print("Testing extraction on first 5 examples:\n")
print("="*80)

for i in range(5):
    example = dataset["train"][i]
    entities = extract_entities_from_one_example(example, label_list)
    
    print(f"\nExample {i+1}:")
    print(f"Review: {' '.join(example['tokens'][:15])}...")  # Show first 15 words
    print(f"Actors: {entities['actors']}")
    print(f"Directors: {entities['directors']}")
    print(f"Sentiment: {'Positive' if example['sentiment'] == 1 else 'Negative'}")
    print("-" * 80)

## Step 6: Counting Actors/Directors by Sentiment

Now we need to count how many times each actor/director appears in positive vs negative reviews.

We'll use `defaultdict` which automatically creates the nested dictionary structure we need:

In [ ]:
# Create storage for counts using defaultdict
# This automatically creates {"positive": 0, "negative": 0} for new entries
actor_sentiment = defaultdict(lambda: {"positive": 0, "negative": 0})
director_sentiment = defaultdict(lambda: {"positive": 0, "negative": 0})

# Process FIRST 50 examples (for testing - later you'll do all of them)
print("Processing first 50 examples...\n")

for i in range(50):
    example = dataset["train"][i]
    entities = extract_entities_from_one_example(example, label_list)
    
    # Determine sentiment
    sentiment_label = "positive" if example['sentiment'] == 1 else "negative"
    
    # Count each actor
    for actor in entities['actors']:
        actor_sentiment[actor][sentiment_label] += 1
    
    # Count each director
    for director in entities['directors']:
        director_sentiment[director][sentiment_label] += 1

print(f"Found {len(actor_sentiment)} unique actors")
print(f"Found {len(director_sentiment)} unique directors")

# Show some results
print("\nFirst 5 actors and their counts:")
for actor, counts in list(actor_sentiment.items())[:5]:
    total = counts['positive'] + counts['negative']
    print(f"  {actor}: {counts['positive']} positive, {counts['negative']} negative (total: {total})")

## Step 7: Finding Top 3 Actors/Directors

To find who is most associated with positive/negative films, we calculate the **proportion** of their appearances that were positive:

```
positive_proportion = positive_count / (positive_count + negative_count)
```

A proportion close to 1.0 (100%) means mostly positive films.
A proportion close to 0.0 (0%) means mostly negative films.

In [ ]:
def calculate_proportions(entity_sentiment_dict):
    """
    Calculate positive proportion for each entity.
    Returns a list of dictionaries with name, counts, and proportion.
    """
    results = []
    
    for entity_name, counts in entity_sentiment_dict.items():
        positive = counts['positive']
        negative = counts['negative']
        total = positive + negative
        
        if total > 0:  # Avoid division by zero
            positive_proportion = positive / total
            results.append({
                'name': entity_name,
                'positive': positive,
                'negative': negative,
                'total': total,
                'positive_proportion': positive_proportion
            })
    
    return results


# Calculate proportions
actor_results = calculate_proportions(actor_sentiment)
director_results = calculate_proportions(director_sentiment)

print(f"Calculated proportions for {len(actor_results)} actors")
print(f"Calculated proportions for {len(director_results)} directors")

### Find Top 3 Actors in POSITIVE Films

In [ ]:
# Sort by positive proportion (highest first)
actor_results_sorted = sorted(actor_results, key=lambda x: x['positive_proportion'], reverse=True)

print("TOP 3 ACTORS MOST LIKELY TO APPEAR IN POSITIVE FILMS")
print("="*70)
for i in range(min(3, len(actor_results_sorted))):
    actor = actor_results_sorted[i]
    print(f"\n{i+1}. {actor['name']}")
    print(f"   Positive: {actor['positive']}, Negative: {actor['negative']}, Total: {actor['total']}")
    print(f"   Positive Proportion: {actor['positive_proportion']:.2%}")

### Find Top 3 Actors in NEGATIVE Films

In [ ]:
# Sort by positive proportion (lowest first) - these are in negative films
actor_results_sorted = sorted(actor_results, key=lambda x: x['positive_proportion'])

print("TOP 3 ACTORS MOST LIKELY TO APPEAR IN NEGATIVE FILMS")
print("="*70)
for i in range(min(3, len(actor_results_sorted))):
    actor = actor_results_sorted[i]
    print(f"\n{i+1}. {actor['name']}")
    print(f"   Positive: {actor['positive']}, Negative: {actor['negative']}, Total: {actor['total']}")
    print(f"   Positive Proportion: {actor['positive_proportion']:.2%}")

### Find Top 3 Directors in POSITIVE Films

In [ ]:
# Sort directors by positive proportion (highest first)
director_results_sorted = sorted(director_results, key=lambda x: x['positive_proportion'], reverse=True)

print("TOP 3 DIRECTORS MOST LIKELY TO DIRECT POSITIVE FILMS")
print("="*70)
for i in range(min(3, len(director_results_sorted))):
    director = director_results_sorted[i]
    print(f"\n{i+1}. {director['name']}")
    print(f"   Positive: {director['positive']}, Negative: {director['negative']}, Total: {director['total']}")
    print(f"   Positive Proportion: {director['positive_proportion']:.2%}")

### Find Top 3 Directors in NEGATIVE Films

In [ ]:
# Sort directors by positive proportion (lowest first)
director_results_sorted = sorted(director_results, key=lambda x: x['positive_proportion'])

print("TOP 3 DIRECTORS MOST LIKELY TO DIRECT NEGATIVE FILMS")
print("="*70)
for i in range(min(3, len(director_results_sorted))):
    director = director_results_sorted[i]
    print(f"\n{i+1}. {director['name']}")
    print(f"   Positive: {director['positive']}, Negative: {director['negative']}, Total: {director['total']}")
    print(f"   Positive Proportion: {director['positive_proportion']:.2%}")

## Step 8: Putting It All Together - Complete Solution

Now that you understand each step, here's the complete solution processing ALL training examples:

In [ ]:
print("Processing ALL training examples...")
print(f"Total examples to process: {len(dataset['train'])}\n")

# Reset our counters
actor_sentiment = defaultdict(lambda: {"positive": 0, "negative": 0})
director_sentiment = defaultdict(lambda: {"positive": 0, "negative": 0})

# Process ALL examples
for i in range(len(dataset["train"])):
    example = dataset["train"][i]
    entities = extract_entities_from_one_example(example, label_list)
    
    sentiment_label = "positive" if example['sentiment'] == 1 else "negative"
    
    # Count actors
    for actor in entities['actors']:
        actor_sentiment[actor][sentiment_label] += 1
    
    # Count directors
    for director in entities['directors']:
        director_sentiment[director][sentiment_label] += 1

print(f"Processing complete!")
print(f"Found {len(actor_sentiment)} unique actors")
print(f"Found {len(director_sentiment)} unique directors")

In [ ]:
# Calculate proportions for all entities
actor_results = calculate_proportions(actor_sentiment)
director_results = calculate_proportions(director_sentiment)

print("Calculated proportions for all entities!")

### Final Results: All Four Categories

In [ ]:
print("\n" + "="*80)
print("FINAL RESULTS - PROCESSING ALL TRAINING DATA")
print("="*80)

# Top 3 actors in positive films
actor_results_pos = sorted(actor_results, key=lambda x: x['positive_proportion'], reverse=True)
print("\nTOP 3 ACTORS MOST LIKELY TO APPEAR IN POSITIVE FILMS")
print("-"*80)
for i in range(min(3, len(actor_results_pos))):
    actor = actor_results_pos[i]
    print(f"{i+1}. {actor['name']}")
    print(f"   Positive: {actor['positive']}, Negative: {actor['negative']}, Total: {actor['total']}")
    print(f"   Positive Proportion: {actor['positive_proportion']:.2%}\n")

# Top 3 actors in negative films
actor_results_neg = sorted(actor_results, key=lambda x: x['positive_proportion'])
print("\nTOP 3 ACTORS MOST LIKELY TO APPEAR IN NEGATIVE FILMS")
print("-"*80)
for i in range(min(3, len(actor_results_neg))):
    actor = actor_results_neg[i]
    print(f"{i+1}. {actor['name']}")
    print(f"   Positive: {actor['positive']}, Negative: {actor['negative']}, Total: {actor['total']}")
    print(f"   Positive Proportion: {actor['positive_proportion']:.2%}\n")

# Top 3 directors in positive films
director_results_pos = sorted(director_results, key=lambda x: x['positive_proportion'], reverse=True)
print("\nTOP 3 DIRECTORS MOST LIKELY TO DIRECT POSITIVE FILMS")
print("-"*80)
for i in range(min(3, len(director_results_pos))):
    director = director_results_pos[i]
    print(f"{i+1}. {director['name']}")
    print(f"   Positive: {director['positive']}, Negative: {director['negative']}, Total: {director['total']}")
    print(f"   Positive Proportion: {director['positive_proportion']:.2%}\n")

# Top 3 directors in negative films
director_results_neg = sorted(director_results, key=lambda x: x['positive_proportion'])
print("\nTOP 3 DIRECTORS MOST LIKELY TO DIRECT NEGATIVE FILMS")
print("-"*80)
for i in range(min(3, len(director_results_neg))):
    director = director_results_neg[i]
    print(f"{i+1}. {director['name']}")
    print(f"   Positive: {director['positive']}, Negative: {director['negative']}, Total: {director['total']}")
    print(f"   Positive Proportion: {director['positive_proportion']:.2%}\n")

print("="*80)
print("COMPLETE!")
print("="*80)

## Summary

### What We Did:

1. **Loaded the data** and understood the structure (tokens, ner_tags, sentiment)
2. **Decoded the BIO tags** to understand what each number means
3. **Extracted complete names** by combining B- and I- tags
4. **Counted appearances** for each actor/director in positive vs negative reviews
5. **Calculated proportions** to see who is most associated with positive/negative films
6. **Found the top 3** in each category

### Key Concepts:

- **Token list**: Just a list of words `['Tom', 'Hanks']`
- **Extract**: Find complete entity names from BIO tags
- **Storage**: Dictionaries mapping names to counts `{"Tom Hanks": {"positive": 5, "negative": 2}}`
- **Proportion**: `positive_count / total_count` tells us tendency toward positive or negative

### Tips for Your Assignment:

1. **Start small**: Test on 10 examples first
2. **Print intermediate results**: Make sure entities are extracted correctly
3. **Check edge cases**: What if a review has no actors? What if a name is at the end?
4. **Scale up gradually**: Once working on 10, try 100, then all

### Common Mistakes to Avoid:

- Forgetting to save the last entity (when review ends with a name)
- Not resetting `current_entity` when you hit an 'O' tag
- Dividing by zero when calculating proportions

You've got this! 🎉